In [10]:
# Base Data Science snippet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from tqdm import tqdm_notebook
import sys
sys.path.append("../")
from delivery import *
from delivery import run_episode, run_n_episodes
from Env import DeliveryEnvironment
from qagent import QAgent
import pymongo
from route_db import Storage
import datetime


%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Points of interest and config

In [13]:
n_points = 30
max_box = 10
n_stops = 20
n_samples = 10**6

pois = np.random.rand(n_points, 2) * max_box
print(pois)

[[6.37627761 9.36069171]
 [5.2728741  6.33573409]
 [7.87184725 2.45322978]
 [8.20597924 6.85975051]
 [9.19737374 0.045247  ]
 [7.01842105 8.83379159]
 [4.34157087 2.63479827]
 [1.4691401  7.86699453]
 [6.53900641 9.30090494]
 [1.19013613 8.86166912]
 [5.23163392 7.28775779]
 [6.32604159 4.15196153]
 [2.13243812 3.65430861]
 [9.27572609 4.09896096]
 [7.29376125 9.05261845]
 [6.46976183 9.65696539]
 [1.13973723 7.51899552]
 [8.92537454 3.76858706]
 [9.92113902 3.90436677]
 [3.62973281 3.269262  ]
 [1.29156479 3.7845213 ]
 [1.95462002 7.65224347]
 [3.43482039 3.48413985]
 [4.30636188 7.11443934]
 [1.02319178 7.7253935 ]
 [5.77993665 4.48957646]
 [0.66218769 1.53621849]
 [1.63448574 3.6807565 ]
 [9.62439313 1.19268552]
 [4.28568983 0.90234382]]


### Simple TSP route gen

In [14]:
dt = datetime.datetime.now().strftime("%d-%H-%M-%S")
# enable database 
st_table = "TSP20Gen"+dt
print("Table: ", st_table)
db = Storage(st_table)

for i in range(n_samples):
    if i % 10000 == 0:
        print("Test gen {} out of {}".format(i, n_samples))
    task_indices = np.random.choice(n_points, n_stops, replace=False)
    reverse_task_ind = np.column_stack((np.arange(n_stops), task_indices))
    task_locations = pois[task_indices, :]
    # no does_task_exist check
    env = DeliveryEnvironment(n_stops = n_stops, method = "distance", poi = task_locations)
    agent = QAgent(nA=env.action_space, nS=env.observation_space, alpha=0.8, gamma=0.8, eps_start=1, eps_decay=0.95, eps_min=0.01, sars='max')
    env, agent, reward = gen_n_episodes(env,agent, n_episodes=500)
    #env.render()
    route_indices = env.stops # need a getter?
    global_route_indices = reverse_task_ind[route_indices,1]
    # add to the database
    
    route_dict = {"input_graph": ','.join(map(str, task_indices.tolist())), "route": ','.join(map(str, global_route_indices.tolist())), 
                  "reward": reward}
    res = db.insert_route(route_dict)
    

print('done')

Table:  TSP20Gen23-23-53-04
Test gen 0 out of 1000000


KeyboardInterrupt: 

In [9]:
# todo
# generate CSV
db.print_table(limit=100)
#csv_path = "path" 
#db.generate_csv_from_table(st_table, csv_path)


{'_id': ObjectId('5d893d44f5a2b952951353de'), 'input_graph': '10,23,26,0,13,7,2,16,12,8,29,3,9,6,18,21,19,20,17,27', 'route': '9,27,10,20,26,8,29,3,6,17,12,16,7,2,23,18,19,21,13,0', 'reward': -51.05625078123217}
{'_id': ObjectId('5d893d44f5a2b952951353df'), 'input_graph': '1,27,24,11,4,8,6,15,21,28,16,17,9,12,29,3,26,20,18,25', 'route': '8,1,29,15,17,6,12,3,24,4,16,21,18,25,11,26,20,28,27,9', 'reward': -36.81897587662199}
{'_id': ObjectId('5d893d45f5a2b952951353e0'), 'input_graph': '27,21,25,7,22,23,13,5,28,14,26,19,9,29,1,17,6,15,0,18', 'route': '22,9,27,28,26,29,1,25,19,18,21,23,7,13,0,14,6,17,15,5', 'reward': -37.394567597625134}
{'_id': ObjectId('5d893d61f5a2b952951353e2'), 'input_graph': '16,26,4,23,13,8,1,12,14,21,11,3,0,6,17,5,7,9,29,25', 'route': '9,0,12,14,6,17,5,29,1,8,25,11,23,21,16,7,4,13,26,3', 'reward': -35.73242491987317}
{'_id': ObjectId('5d893d61f5a2b952951353e3'), 'input_graph': '18,17,23,1,8,15,10,5,3,29,7,12,24,28,14,9,2,6,11,0', 'route': '10,28,3,29,1,8,11,18,23,2,